## Advanced Lane Finding Project

The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.

---

### 1. Import libraries.

In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import ntpath
import math
import pickle
import os
import pandas as pd
from moviepy.editor import VideoFileClip
from IPython.display import HTML

### 1. Create utils.

In [2]:
class Util:
    
    
    def get_fname(path):
        
        _, tail = ntpath.split(path)
        
        return tail
    
    
    def set_output(number, output = None):
        
        if output == 'qt':
            
            %matplotlib qt
            fontsize = 10
            nrows = int(math.sqrt(number))
            ncols = number//nrows
            figure, axes = plt.subplots(nrows, ncols, figsize=(10*ncols, 6*math.ceil(number/ncols)))
            
            return output, figure, axes, fontsize, ncols
        
        elif output == 'inline':
            
            %matplotlib inline
            fontsize = 18
            ncols = 2
            figure, axes = plt.subplots(number//2,ncols, figsize=(10*ncols, 6*number//2))
            
            return output, figure, axes, fontsize, ncols
        
        elif output != None and output != '' and output[-1] != '/':
            
            output += '/'
            
            return output, None, None, None, None
    
    
    def show_output(output, figure, title):
        
        if output == 'qt':
            
            figure.canvas.set_window_title(title)
            figure.patch.set_facecolor('xkcd:gray')
            plt.subplots_adjust(top=0.95, bottom=0.05, hspace=0.3, wspace=0.3)
            
        elif output == 'inline':
            
            figure.suptitle(title, fontsize=18)
            plt.subplots_adjust(top=4, hspace=4)
            plt.tight_layout()
            
        
        if output == 'inline' or output == 'qt':
            
            plt.show()
            

### 3. Create calibration tools.

In [3]:
class Camera:
        
        
    def __init__(self, calibrationset=None):
        
        if calibrationset == None: 
            
            dist_pickle = pickle.load(open("camera_calibration.p", "rb"))
            self.mtx = dist_pickle["mtx"]
            self.dist = dist_pickle["dist"]
            self.images = []
            
        else:
            
            self.mtx = []
            self.dist = []
            self.images = glob.glob(calibrationset)
    
    
    def calibrate(self, output=None):
        
        if (len(self.images) == 0):
            
            return False;
        
        objp = np.zeros((6*9,3), np.float32)
        objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)      
        counter = 0 
        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image plane.
        testimages = []
        
        output, figure, axes, fontsize, ncols = Util.set_output(len(self.images), output)
        
        for fname in self.images:
            
            img = cv2.imread(fname)
            gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, (9,6),None)
            if ret == True:
                objpoints.append(objp)
                imgpoints.append(corners)

                # Draw and display the corners
                img = cv2.drawChessboardCorners(img, (9,6), corners, ret)
                if output == 'inline' or output == 'qt':
                    
                    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                    axes[counter//ncols][counter%ncols].set_title(Util.get_fname(fname), fontsize=fontsize)
                    axes[counter//ncols][counter%ncols].imshow(img);
                    counter += 1
                    
                elif output != None:
                    
                    cv2.imwrite(output+'corners_'+Util.get_fname(fname), img)
            else:
                testimages.append(fname)
                
        if (len(imgpoints) == 0):
            
            return False
        
        ret, self.mtx, self.dist, _, _ = cv2.calibrateCamera(objpoints, imgpoints, img.shape[1::-1], None, None)
        
        # Return value from calibrateCamera is more than 1 which suggest poor quality calibration
        if (ret > 1.1):
            
            return False
        
        dist_pickle = {}
        dist_pickle["mtx"] = self.mtx
        dist_pickle["dist"] = self.dist
        pickle.dump(dist_pickle, open("camera_calibration.p", "wb"))
        
        if (output == None):
            
            return True
        
        if (len(testimages) == 0):
            
            testimages.append(images[0])
        
        img = cv2.imread(testimages[0])
        dst = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        if output == 'inline' or output == 'qt':
            
            counter += 1
            axes[counter//ncols][counter%ncols].set_title("Test image", fontsize=fontsize)
            axes[counter//ncols][counter%ncols].imshow(img);
            counter += 1
            axes[counter//ncols][counter%ncols].set_title("Undistorted image", fontsize=fontsize)
            axes[counter//ncols][counter%ncols].imshow(dst)
            
            Util.show_output(output, figure, 'Calibration')
            
        elif output != None:

            cv2.imwrite(output + 'corners_undistort_' + Util.get_fname(testimages[0]), dst)

            
    def undistort(self, img = [], fname=None, output=None):
        
        if len(self.mtx) == 0 or len(self.dist) == 0:
            return None
        if fname != None and len(img) == 0:
            img = cv2.imread(fname)
        if len(img) == 0:
            return None
        undistort = cv2.undistort(img, self.mtx, self.dist, None, self.mtx) 
        if output != None and output != '' and output[-1] != '/':
            output += '/'
        if output != None:
            cv2.imwrite(output + 'undistort_' + Util.get_fname(fname), undistort)
            
        return img, undistort
    
    
    def test(self, files, output):
        
        output, figure, axes, fontsize, ncols = Util.set_output(len(files)*2, output)
        
        counter = 0
        
        for file in files:
            
            original, undistort = camera.undistort(fname=file, output=output)
            
            if output == 'qt' or output == 'inline':
                original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
                axes[counter//ncols][counter%ncols].set_title(Util.get_fname(file), fontsize=fontsize)
                axes[counter//ncols][counter%ncols].imshow(original)
                counter+=1
                undistort = cv2.cvtColor(undistort, cv2.COLOR_BGR2RGB)
                axes[counter//ncols][counter%ncols].set_title('Undistorted Image', fontsize=fontsize)
                axes[counter//ncols][counter%ncols].imshow(undistort)
                counter+=1
            
        Util.show_output(output, figure, 'Original vs. Undistorted Test Images')
        

### 4. Make camera calibration based on calibration images set and then undistort test images.

In [4]:
files = glob.glob('output_images/undistort*')
files.extend(glob.glob('output_images/corners*'))
for file in files:
    os.remove(file)

camera = Camera('camera_cal/calibration*.jpg')
camera.calibrate('output_images')
files = glob.glob('test_images/*')

camera.test(files, 'output_images')

### 5. Create transform tools.

In [5]:
class Transform:
    
    # Perspective transformation parameters
    src = np.float32([(230, 690), (545, 480), (735, 480), (1050, 690)])
    dst = np.float32([(230, 720), (230,   0), (1050,  0), (1050, 720)])
    
    
    def read(fname):
        
        return cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB);
    
    
    def write(binary, fname=None, output=None):
        
        if output != None and output != '' and output[-1] != '/':
            output += '/'
        elif output == None:
            output = ''
        if fname != None:
            cv2.imwrite(output+fname, binary*255)
    
    
    def abs_threshold(img, fname=None, output=None, orient='x', sobel_kernel=3, thresh=(0, 255)):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        if orient == 'x':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        elif orient == 'y':
            sobel = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        else:
            return None
        abs_sobel = np.absolute(sobel)
        scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
        binary = np.zeros_like(scaled_sobel)
        binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
        if fname != None:
            Transform.write(binary, 'transform_abs_'+fname, output)
            
        return binary
    
    
    def mag_threshold(img, fname=None, output=None, sobel_kernel=3, thresh=(0, 255)):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        abs_sobelxy = np.sqrt(np.power(sobelx,2) + np.power(sobely, 2))
        scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
        binary = np.zeros_like(scaled_sobel)
        binary[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1
        if fname != None:
            Transform.write(binary, 'transform_mag_'+fname, output)
            
        return binary
    
    
    def dir_threshold(img, fname=None, output=None, sobel_kernel=3, thresh=(0, np.pi/2)):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        sobelx = cv2.Sobel(img, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(img, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
        abs_sobelx = np.absolute(sobelx)
        abs_sobely = np.absolute(sobely)
        direction = np.arctan2(abs_sobely, abs_sobelx)
        binary = np.zeros_like(direction)
        binary[(direction >= thresh[0]) & (direction <= thresh[1])] = 1
        if fname != None:
            Transform.write(binary, 'transform_dir_'+fname, output)
            
        return binary
    
    
    def gray_threshold(img, fname=None, output=None, thresh=(0, 255)):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        binary = np.zeros_like(gray)
        binary[(gray > thresh[0]) & (gray <= thresh[1])] = 1
        if fname != None:
            Transform.write(binary, 'transform_gray_'+fname, output)
            
        return binary
    
    
    def color_threshold(img, fname=None, output=None, space='rgb', select='r', thresh=(0, 255)):
        
        if space == 'hls':
            image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif space == 'hsv':
            image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif output != 'rgb':
            return None
        C = image[:,:,space.find(select)]
        binary = np.zeros_like(C)
        binary[(C > thresh[0]) & (C <= thresh[1])] = 1
        if fname != None:
            Transform.write(binary, 'transform_'+space+'_'+select+'_'+fname, output)
            
        return binary
    
    
    def canny(img, fname=None, output=None, kernel_size=3):
        
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        med = np.median(gray)
        image = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
        image = cv2.Canny(image, (2/3)*med, (3/4)*med)
        binary = np.zeros_like(image)
        binary[(image >= 1)] = 1
        if fname != None:
            Transform.write(binary, 'transform_canny_'+fname, output)
            
        return binary
        
        
    def combine(img, fname=None, output=None):
        
        binary = np.zeros_like(img[0])
        for image in img:
            binary[image == 1] = 1 
        if fname != None:
            Transform.write(binary, 'transform_combine_'+fname, output)
            
        return binary
    
    
    def unwarp(img, fname=None, output=None):
        
        M = cv2.getPerspectiveTransform(Transform.src, Transform.dst)
        
        binary = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
        if fname != None:
            Transform.write(binary, 'transform_unwarp_'+fname, output)
            
        return binary
    
    
    def warp(img, fname=None, output=None):

        M = cv2.getPerspectiveTransform(Transform.dst, Transform.src)
        
        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        result = cv2.warpPerspective(img, M, img.shape[1::-1], flags=cv2.INTER_LINEAR)
        
        if fname != None:
            Transform.write(result, 'transform_warp_'+fname, output)
            
        return result
        
    
    
    def process(img, fname=None, output=None, abs_thresh=(35, 105), hls_s_thresh=(180, 245),
                hls_h_thresh=(20, 45), gray_thresh=(205, 255)):
        
        combined = Transform.combine([Transform.gray_threshold(img, fname=fname, output=output, thresh=gray_thresh),
                                     Transform.abs_threshold(img, fname=fname, output=output, sobel_kernel=7, thresh=abs_thresh),
                                     Transform.color_threshold(img, fname=fname, output=output, space='hls', select='s', thresh=hls_s_thresh),
                                     Transform.color_threshold(img, fname=fname, output=output, space='hls', select='h', thresh=hls_h_thresh)],
                                    fname=fname, output=output)
        unwarped = Transform.unwarp(combined, fname=fname, output=output)
        
        return combined, unwarped
    
    
    def test(files, output=None):
           
        output, figure, axes, fontsize, ncols = Util.set_output(len(files)*2, output)
        counter=0
        
        for file in files:
            
            img = Transform.read(file)
            abs_thresh = (35, 105)
            hls_s_thresh = (180, 245)
            hls_h_thresh = (20, 45)
            gray_thresh = (205, 255)
            
            if output != 'inline' and output != 'qt' and output != None:
                
                name, ext = os.path.splitext(Util.get_fname(file))
                Transform.abs_threshold(img, thresh=abs_thresh,
                                        fname=name+'_x_3_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                Transform.abs_threshold(img, sobel_kernel=5, thresh=abs_thresh,
                                        fname=name+'_x_5_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                Transform.abs_threshold(img, sobel_kernel=7, thresh=abs_thresh,
                                        fname=name+'_x_7_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                Transform.abs_threshold(img, orient='y', thresh=abs_thresh,
                                        fname=name+'_y_3_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                Transform.abs_threshold(img, orient='y', sobel_kernel=5, thresh=abs_thresh,
                                        fname=name+'_y_5_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                Transform.abs_threshold(img, orient='y', sobel_kernel=7, thresh=abs_thresh,
                                        fname=name+'_y_7_'+str(abs_thresh[0])+'_'+str(abs_thresh[1])+ext,
                                        output=output)
                thresh = (30, 100)
                Transform.mag_threshold(img, thresh=thresh,
                                        fname=name+'_3_'+str(thresh[0])+'_'+str(thresh[1])+ext,
                                        output=output)
                Transform.mag_threshold(img, sobel_kernel=5, thresh=thresh,
                                        fname=name+'_5_'+str(thresh[0])+'_'+str(thresh[1])+ext,
                                        output=output)
                Transform.mag_threshold(img, sobel_kernel=7, thresh=thresh,
                                        fname=name+'_7_'+str(thresh[0])+'_'+str(thresh[1])+ext,
                                        output=output)
                thresh = (2/3, 4/3)            
                Transform.dir_threshold(img, thresh=thresh,
                                        fname=name+'_3_'+str(round(thresh[0], 2))+'_'+str(round(thresh[1],2))+ext,
                                        output=output)
                Transform.dir_threshold(img, sobel_kernel=5, thresh=thresh,
                                        fname=name+'_5_'+str(round(thresh[0], 2))+'_'+str(round(thresh[1],2))+ext,
                                        output=output)
                Transform.dir_threshold(img, sobel_kernel=7, thresh=thresh,
                                        fname=name+'_7_'+str(round(thresh[0], 2))+'_'+str(round(thresh[1],2))+ext,
                                        output=output)
                Transform.gray_threshold(img, thresh=gray_thresh,
                                         fname=name+'_'+str(gray_thresh[0])+'_'+str(gray_thresh[1])+ext,
                                         output=output)
                Transform.color_threshold(img, space='hls', select='h', thresh=hls_h_thresh,
                                          fname=name+'_'+str(hls_h_thresh[0])+'_'+str(hls_h_thresh[1])+ext,
                                          output=output)
                thresh = (90, 255)
                Transform.color_threshold(img, space='hls', select='l', thresh=thresh,
                                          fname=name+'_'+str(thresh[0])+'_'+str(thresh[1])+ext,
                                          output=output)
                Transform.color_threshold(img, space='hls', select='s', thresh=hls_s_thresh,
                                          fname=name+'_'+str(hls_s_thresh[0])+'_'+str(hls_s_thresh[1])+ext,
                                          output=output)
                Transform.canny(img, kernel_size=5, fname=Util.get_fname(file), output=output)
                
            combined, unwarped = Transform.process(img, abs_thresh=abs_thresh, hls_s_thresh=hls_s_thresh,
                                                   hls_h_thresh=hls_h_thresh, gray_thresh=gray_thresh,
                                                   fname=Util.get_fname(file), output=output)
            
            if output == 'qt' or output == 'inline':
                
                axes[counter//ncols][counter%ncols].set_title("Combined Image", fontsize=fontsize)
                axes[counter//ncols][counter%ncols].imshow(combined, cmap='gray')
                counter+=1
                axes[counter//ncols][counter%ncols].set_title("Unwarped Image", fontsize=fontsize)
                axes[counter//ncols][counter%ncols].imshow(unwarped, cmap='gray')
                counter+=1
            
        Util.show_output(output, figure, 'Transformed vs. Unwarped Test Images')
            


### 6. Apply transformations on undistorted test images.

In [6]:
files = glob.glob('output_images/transform*')
for file in files:
    os.remove(file)
    
files = glob.glob('output_images/undistort*')

Transform.test(files, 'output_images')

### 7. Create line detection tools.

In [7]:
class Line:
    
    
    def __init__(self, nwindows=9, margin=150, minpix=50, n=5):
        
        self.nwindows = nwindows
        self.margin = margin
        self.minpix = minpix
        self.n = n
        
        # was the line detected in the last iteration?
        self.detected = False
        # x values of the last n fits of the line
        self.recent_xfitted = []
        #average x values of the fitted line over the last n iterations
        self.bestx = None
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = []
        #radius of curvature of the line in some units
        self.radius_of_curvature = None
        #distance in meters of vehicle center from the line
        self.line_base_pos = None
        #difference in fit coefficients between last and new fits
        #self.diffs = np.array([0,0,0], dtype='float') # Not used in this project
        #x values for detected line pixels
        self.allx = None                                                    
        #y values for detected line pixels
        self.ally = None                                                    
        
        self.ploty = []
        
        
    def init(self, binary, base = None):
        
        self.binary = binary
        if base != None:
            self.line_base_pos = base
        
        
    def find_pixels(self):
        
        # Set height of windows - based on nwindows above and image shape
        window_height = np.int(self.binary.shape[0]//self.nwindows)
        
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = self.binary.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Current positions to be updated later for each window in nwindows
        current = self.line_base_pos

        # Create empty lists to receive left and right lane pixel indices
        inds = []

        # Step through the windows one by one
        for window in range(self.nwindows):
            
            # Identify window boundaries in x and y (and right and left)
            win_y_low = self.binary.shape[0] - (window + 1) * window_height
            win_y_high = self.binary.shape[0] - window * window_height
            win_x_low = current - self.margin
            win_x_high = current + self.margin
        
            ### Identify the nonzero pixels in x and y within the window ###
            good_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                         (nonzerox >= win_x_low) &  (nonzerox < win_x_high)).nonzero()[0]

            # Append these indices to the lists
            inds.append(good_inds)
        
            if len(good_inds) > self.minpix:
                current = np.int(np.mean(nonzerox[good_inds]))

        inds = np.concatenate(inds)

        # Extract left and right line pixel positions
        if len(inds) > self.minpix*self.nwindows//2:
            self.allx = nonzerox[inds]
            self.ally = nonzeroy[inds] 
            self.detected = False
        else:
            self.detected = True
            
        return self.detected
    
    
    def fit_polynomial(self):

        if len(self.current_fit) > self.n:
            self.current_fit = self.current_fit[1:]
        self.current_fit.append(np.polyfit(self.ally, self.allx, 2))
        
        self.best_fit = pd.DataFrame(self.current_fit).mean().values

        # Generate x and y values for plotting
        ploty = np.linspace(0, self.binary.shape[0]-1, self.binary.shape[0] )
        try:
            fitx = self.best_fit[0]*ploty**2 + self.best_fit[1]*ploty + self.best_fit[2]
        except TypeError:
            # Avoids an error if `left` and `right_fit` are still none or incorrect
            print('The function failed to fit a line!')
            fitx = 1*ploty**2 + 1*ploty
        
        if len(self.recent_xfitted) > self.n:
            self.recent_xfitted = self.recent_xfitted[1:]
        self.recent_xfitted.append(fitx)
        self.bestx = pd.DataFrame(self.recent_xfitted).mean().values.astype(int)
        self.ploty = ploty    
        
        return self.bestx, ploty
    
    
    def search_around_poly(self, binary):

        # Grab activated pixels
        nonzero = binary.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
    
        poly = (self.best_fit[0]*nonzeroy**2 + self.best_fit[1]*nonzeroy + self.best_fit[2])
        inds = ((nonzerox >= poly - self.margin) & (nonzerox < poly + self.margin)).nonzero()[0]
        
        if len(inds) > self.minpix*self.nwindows//2:
            # Again, extract left and right line pixel positions
            self.allx = nonzerox[inds]
            self.ally = nonzeroy[inds] 
            self.detected = True
        else:
            self.detected = False

        # Fit new polynomials
        return self.detected, self.fit_polynomial()
    
    
    def get_curvature_radius(self, xm_per_pix):
        
        y_eval = np.max(self.ploty)
        #ym_per_pix = 30/720
        # This project uses smaller field of view so the y dimension is smaller as well
        ym_per_pix = 20/720 # meters per pixel in y dimension

        # Fit new polynomials to x,y in world space
        fit_cr = np.polyfit(self.ploty*ym_per_pix, self.bestx*xm_per_pix, 2)
        # Calculate the new radii of curvature
        self.radius_of_curvature = ((1 + (2*fit_cr[0]*y_eval*ym_per_pix + fit_cr[1])**2)**1.5) / np.absolute(2*fit_cr[0])
 
        # Now our radius of curvature is in meters
        return self.radius_of_curvature
    

### 8. Create lane detection tools.

In [13]:
class LaneDetector:
    
    
    def __init__(self, camera, left_line, right_line, output=None):
        
        self.camera = camera
        self.left_line = left_line
        self.right_line = right_line
        self.leftx_base = None
        self.rightx_base = None
        self.output = output
        self.counter = 0
        #self.left_not_detected = 0
        #self.right_not_detected = 0
        #self.left_fitx = []
        #self.right_fitx = []
        #self.ploty = []
        
        
    def read_binary(fname):
        
        return cv2.imread(fname)[:,:,0]//255
    
    
    def find_lane_start(binary):
        
        histogram = np.sum(binary[binary.shape[0]//2:,:], axis=0)
        midpoint = np.int(histogram.shape[0]//2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint
        
        return leftx_base, rightx_base
    
    
    def process(self, img):
        
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        if self.output != None:
            fname = '{:08d}.jpg'.format(self.counter)
        else:
            fname = None
        _, image = self.camera.undistort(img=img, fname=fname, output=self.output)
        #_, image = self.camera.undistort(img=img)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        _, binary = Transform.process(image, fname=fname, output=self.output)
        #_, binary = Transform.process(image)

        if self.leftx_base == None or self.rightx_base == None:
            self.leftx_base, self.rightx_base = LaneDetector.find_lane_start(binary)
            self.left_line.init(binary, self.leftx_base)
            self.right_line.init(binary, self.rightx_base)
            left_detected = self.left_line.find_pixels()
            right_detected = self.right_line.find_pixels()
            #if left_detected == True:
            #    self.left_fitx, self.ploty = self.left_line.fit_polynomial()
            left_fitx, ploty = self.left_line.fit_polynomial()
            #if right_detected == True:
            #    self.right_fitx, self.ploty = self.right_line.fit_polynomial()
            right_fitx, ploty = self.right_line.fit_polynomial()
        else:
            left_detected, (left_fitx, ploty) = self.left_line.search_around_poly(binary)
            #if left_detected == False:
            #    self.left_not_detected += 1
            #else:
            #    self.left_fitx = left_fitx
            #    self.ploty = ploty
            #    self.left_not_detected = 0
            #if self.left_not_detected > 3:
            #    self.leftx_base = None
            right_detected, (right_fitx, ploty) = self.right_line.search_around_poly(binary)
            #if right_detected == False:
            #    self.right_not_detected += 1
            #else:
            #    self.right_fitx = right_fitx
            #    self.ploty = ploty
            #    self.right_not_detected = 0
            #if self.right_not_detected > 3:
            #    self.rightx_base = None
            
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(binary).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))        
        
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
        
        warp = Transform.warp(color_warp)
        
        # Combine the result with the original image
        result = cv2.addWeighted(image, 1, warp, 0.3, 0)
        
        # Width of lane in pixels
        lane_width_in_pix = right_fitx[len(ploty)-1]-left_fitx[len(ploty)-1]
        xm_per_pix = 3.7/lane_width_in_pix
        
        curverad = int(np.mean([self.left_line.get_curvature_radius(xm_per_pix),
                                self.right_line.get_curvature_radius(xm_per_pix)]))
        
        cv2.putText(result, "The lane curvature radius is {}m.".format(curverad), (100, 50),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        
        center = round(3.7*((left_fitx[len(ploty)-1]+right_fitx[len(ploty)-1])//2-image.shape[1]//2)/lane_width_in_pix,2)
        
        if center > 0:
            msg = "The car is located {:.2f}m to the left from the center of the lane.".format(center)
        elif center < 0:
            msg = "The car is located {:.2f}m to the right from the center of the lane.".format(abs(center))
        else:
            msg = "The car is located at the center of the lane."
        
        cv2.putText(result, msg, (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

        if self.output != None and self.output != '' and self.output[-1] != '/':
            self.output += '/'
            
        if self.output != None:
            cv2.imwrite(self.output+fname, cv2.cvtColor(result, cv2.COLOR_BGR2RGB))
            
        self.counter += 1
        
        return result
        
        
    def test(files, output=None):
        
        output, figure, axes, fontsize, ncols = Util.set_output(len(files), output)
        counter = 0
        
        for file in files:
            binary = LaneDetector.read_binary(file)
            leftx_base, rightx_base = LaneDetector.find_lane_start(binary)
            left_line = Line()
            left_line.init(binary, leftx_base)
            left_line.find_pixels()
            left_fitx, ploty = left_line.fit_polynomial()
            right_line = Line()
            right_line.init(binary, rightx_base)
            right_line.find_pixels()
            right_fitx, ploty = right_line.fit_polynomial()
            
            if output == 'qt' or output == 'inline':
                
                axes[counter//ncols][counter%ncols].set_title("Detected Lane", fontsize=fontsize)
                axes[counter//ncols][counter%ncols].imshow(binary, cmap='gray')
                axes[counter//ncols][counter%ncols].plot(left_fitx, ploty, color='green')
                axes[counter//ncols][counter%ncols].plot(right_fitx, ploty, color='green')
                counter+=1
                
            elif output != None:
                
                out_img = np.dstack((binary, binary, binary))*255
                pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
                pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
                cv2.polylines(out_img, np.int_([pts_left]), False, (0,255, 0), thickness=3)
                cv2.polylines(out_img, np.int_([pts_right]), False, (0,255, 0), thickness=3)
                cv2.imwrite(output+'detect_'+Util.get_fname(file), out_img) 
        
        Util.show_output(output, figure, 'Detected Lanes')
        

### 9. Detect lanes.

In [14]:
files = glob.glob('output_images/detect*')
for file in files:
    os.remove(file)

files = glob.glob('output_images/transform_unwarp*')

LaneDetector.test(files, 'output_images')

### 10. Test short video.

In [15]:
detector = LaneDetector(Camera(), Line(), Line(), 'output_videos')

video = 'project_video.mp4'
video_output = 'output_videos/short_'+video

files = glob.glob('output_videos/*.jpg')
for file in files:
    os.remove(file)

if os.path.isfile(video_output):
    os.remove(video_output)

short_clip = VideoFileClip(video).subclip(0,1)
short_clip_output = short_clip.fl_image(detector.process)
%time short_clip_output.write_videofile(video_output, audio=False)


t:   0%|          | 0/25 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/short_project_video.mp4.
Moviepy - Writing video output_videos/short_project_video.mp4



Moviepy - Done !
Moviepy - video ready output_videos/short_project_video.mp4
CPU times: user 6.24 s, sys: 361 ms, total: 6.6 s
Wall time: 6.12 s


### 11. Test project video.

In [16]:
detector = LaneDetector(Camera(), Line(), Line())

video = 'project_video.mp4'
video_output = 'output_videos/'+video

if os.path.isfile(video_output):
    os.remove(video_output)

clip = VideoFileClip(video)
clip_output = clip.fl_image(detector.process)
%time clip_output.write_videofile(video_output, audio=False)


t:   0%|          | 0/1260 [00:00<?, ?it/s, now=None]

Moviepy - Building video output_videos/project_video.mp4.
Moviepy - Writing video output_videos/project_video.mp4



Moviepy - Done !
Moviepy - video ready output_videos/project_video.mp4
CPU times: user 4min 36s, sys: 5.36 s, total: 4min 41s
Wall time: 3min 45s


### 12. See the result.

In [17]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))